[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/camenduru/HandRefiner-colab/blob/main/HandRefiner_gradio_colab.ipynb)

In [ ]:
%cd /content
!git clone -b dev https://github.com/camenduru/HandRefiner
%cd /content/HandRefiner

!pip install -q pytorch-lightning omegaconf einops yacs trimesh mediapipe rtree boto3 gradio==3.50.2
!pip install -q git+https://gitlab.eecs.umich.edu/ngv-python-modules/opendr
!pip install -q https://download.pytorch.org/whl/cu121/xformers-0.0.22.post7-cp310-cp310-manylinux2014_x86_64.whl
!apt -y install -qq aria2

%cd /content/HandRefiner
!git clone -b dev --recursive https://github.com/camenduru/MeshGraphormer
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/camenduru/HandRefiner/resolve/main/basicModel_neutral_lbs_10_207_0_v1.0.0.pkl -d /content/HandRefiner/MeshGraphormer/src/modeling/data -o basicModel_neutral_lbs_10_207_0_v1.0.0.pkl
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/camenduru/HandRefiner/resolve/main/MANO_RIGHT.pkl -d /content/HandRefiner/MeshGraphormer/src/modeling/data -o MANO_RIGHT.pkl
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/camenduru/HandRefiner/resolve/main/graphormer_release/graphormer_3dpw_state_dict.bin -d /content/HandRefiner/MeshGraphormer/models/graphormer_release -o graphormer_3dpw_state_dict.bin
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/camenduru/HandRefiner/resolve/main/graphormer_release/graphormer_h36m_state_dict.bin -d /content/HandRefiner/MeshGraphormer/models/graphormer_release -o graphormer_h36m_state_dict.bin
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/camenduru/HandRefiner/resolve/main/graphormer_release/graphormer_hand_state_dict.bin -d /content/HandRefiner/MeshGraphormer/models/graphormer_release -o graphormer_hand_state_dict.bin
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/camenduru/HandRefiner/raw/main/hrnet/cls_hrnet_w40_sgd_lr5e-2_wd1e-4_bs32_x100.yaml -d /content/HandRefiner/MeshGraphormer/models/hrnet -o cls_hrnet_w40_sgd_lr5e-2_wd1e-4_bs32_x100.yaml
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/camenduru/HandRefiner/raw/main/hrnet/cls_hrnet_w64_sgd_lr5e-2_wd1e-4_bs32_x100.yaml -d /content/HandRefiner/MeshGraphormer/models/hrnet -o cls_hrnet_w64_sgd_lr5e-2_wd1e-4_bs32_x100.yaml
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/camenduru/HandRefiner/resolve/main/hrnet/hrnetv2_w40_imagenet_pretrained.pth -d /content/HandRefiner/MeshGraphormer/models/hrnet -o hrnetv2_w40_imagenet_pretrained.pth
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/camenduru/HandRefiner/resolve/main/hrnet/hrnetv2_w64_imagenet_pretrained.pth -d /content/HandRefiner/MeshGraphormer/models/hrnet -o hrnetv2_w64_imagenet_pretrained.pth
%cd /content/HandRefiner/MeshGraphormer
!pip install -e .
!pip install -e ./manopth/.

!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/camenduru/HandRefiner/resolve/main/hand_landmarker.task -d /content/HandRefiner/preprocessor -o hand_landmarker.task
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/camenduru/HandRefiner/resolve/main/inpaint_depth_control.ckpt -d /content/HandRefiner/models -o inpaint_depth_control.ckpt
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/camenduru/HandRefiner/resolve/main/ckpt200-multisc-pred.zip -d /content/HandRefiner/MeshGraphormer/predictions -o ckpt200-multisc-pred.zip
!mkdir /content/HandRefiner/output
%cd /content/HandRefiner

import gradio as gr
import subprocess

def generate(image, prompt, seed):
    print(image, prompt, seed)
    command = f"python handrefiner.py --input_img {image} --out_dir /content/HandRefiner/output --strength 0.55 --weights /content/HandRefiner/models/inpaint_depth_control.ckpt --prompt '{prompt}' --seed {seed}"
    try:
        result = subprocess.run(command, shell=True, check=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)
        output_path = '/content/HandRefiner/output/image_0.jpg'
        print("Output:", result.stdout)
        return output_path
    except subprocess.CalledProcessError as e:
        print("Error:", e.stderr)
        return None

with gr.Blocks() as demo:
    with gr.Row():
      with gr.Column():
          image = gr.Image(type='filepath')
          textbox = gr.Textbox(show_label=False, value="a person facing the camera, making a hand gesture, indoor")
          seed = gr.Slider(minimum=0, maximum=1000000, value=643534)
          button = gr.Button()
      output_image = gr.Image(show_label=False, type="filepath", interactive=False, height=512, width=512)
    button.click(fn=generate, inputs=[image, textbox, seed], outputs=[output_image])

demo.queue().launch(inline=False, share=True, debug=True)